In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import re
import shutil
import pandas as pd
import scipy.stats
import scipy
from functools import reduce
import exdir
import expipe
from distutils.dir_util import copy_tree
import septum_mec
import spatial_maps as sp
import head_direction.head as head
import septum_mec.analysis.data_processing as dp
import septum_mec.analysis.registration
from septum_mec.analysis.plotting import despine, savefig, plot_bootstrap_timeseries
from spatial_maps.fields import find_peaks, calculate_field_centers, separate_fields_by_laplace
from spike_statistics.core import permutation_resampling_test, block_bootstrap
import speed_cells.speed as spd
from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from septum_mec.analysis.statistics import load_data_frames, make_paired_tables, make_statistics_table

/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


In [2]:
project_path = dp.project_path()
project = expipe.get_project(project_path)
actions = project.actions

output_path = pathlib.Path("output") / "longitudinal-comparisons"
(output_path / "statistics").mkdir(exist_ok=True, parents=True)
(output_path / "figures").mkdir(exist_ok=True, parents=True)

# Load cell statistics and shuffling quantiles

In [3]:
data, labels, colors, queries = load_data_frames()

Number of sessions above threshold 194
Number of animals 4
Number of individual gridcells 139
Number of gridcell recordings 230


# Analysis

In [4]:
max_speed = .5 # m/s only used for speed score
min_speed = 0.02 # m/s only used for speed score
position_sampling_rate = 100 # for interpolation
position_low_pass_frequency = 6 # for low pass filtering of position

box_size = [1.0, 1.0]
bin_size = 0.02
smoothing_low = 0.03
smoothing_high = 0.06

speed_binsize = 0.02

stim_mask = True
# baseline_duration = 600
baseline_duration = None

In [5]:
data_loader = dp.Data(
    position_sampling_rate=position_sampling_rate, 
    position_low_pass_frequency=position_low_pass_frequency,
    box_size=box_size, bin_size=bin_size, 
    stim_mask=stim_mask, baseline_duration=baseline_duration
)

In [6]:
def fftcorrelate2d(arr1, arr2, normalize=False, **kwargs):
    from copy import copy
    arr1 = copy(arr1)
    arr2 = copy(arr2)
    from astropy.convolution import convolve_fft
    if normalize:
        # https://stackoverflow.com/questions/53436231/normalized-cross-correlation-in-python
        a_ = arr1.ravel()
        v_ = arr2.ravel()
        arr1 = (arr1 - np.mean(a_)) / (np.std(a_) * len(a_))
        arr2 = (arr2 - np.mean(v_)) / np.std(v_)
    corr = convolve_fft(arr1, np.fliplr(np.flipud(arr2)), normalize_kernel=False, **kwargs)
    return corr


def cross_correlation_distance(r1, r2):
    r12 = fftcorrelate2d(r1, r2)
    labels = separate_fields_by_laplace(r12, threshold=0)
    peaks = calculate_field_centers(r12, labels)
    centered_peaks = peaks - np.array(r1.shape) / 2
    offset = np.linalg.norm(centered_peaks, axis=1)
    distance_idx = np.argmin(offset)
    distance = offset[distance_idx]
    angle = np.arctan2(*centered_peaks[distance_idx])
    
    return distance, angle


def cross_correlation_centre_of_mass(r1, r2):
    from scipy import ndimage
    r12 = fftcorrelate2d(r1, r2)
    cntr = ndimage.center_of_mass(r12)
    
    centered_cntr = cntr - np.array(r1.shape) / 2
    distance = np.linalg.norm(centered_cntr)
    angle = np.arctan2(*centered_cntr)
    
    return distance, angle

In [56]:
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']
labels = [
    'Baseline Ia', 
    'Baseline Ib', 
    'Baseline I', 
    '11 Hz', 
    'Baseline IIa',
    'Baseline IIb',
    'Baseline II', 
    '30 Hz'
]
queries = [
    'baseline and i and Hz11',
    'baseline and i and Hz11',
    'baseline and i and Hz11',
    'frequency==11 and stim_location=="ms"', 
    'baseline and ii and Hz30',
    'baseline and ii and Hz30',
    'baseline and ii and Hz30',
    'frequency==30 and stim_location=="ms"']

In [64]:
keys = [
    'gridness',
    'speed_score',
    'max_rate',
    'average_rate',
    'action',
    'unit_name'
]

cell_types = [
    'gridcell',
    'ns_inhibited', 
    'ns_not_inhibited',
    'bs',
    'bs_not_gridcell'
]

results, labels = make_paired_tables(data, keys, cell_types=cell_types, queries=queries, labels=labels)

In [71]:
for cell_type in cell_types:
#     results[cell_type]['action'] = resultss[cell_type]['action']
#     results[cell_type]['unit_name'] = resultss[cell_type]['unit_name']
    results[cell_type]['xcorr_cntr_mass'] = results[cell_type]['action'].copy()

In [62]:
results['gridcell']['action'].groupby('entity').count()

,unit_idnum,channel_group,date,Baseline I,11 Hz,Baseline II,30 Hz
entity,,,,,,,
1833,94,94,94,43,38,36,24
1834,15,15,15,8,8,4,3
1839,20,20,20,9,6,5,6
1849,8,8,8,3,4,1,2


In [17]:
diff_labels = [
    ['Baseline Ia', 'Baseline Ib'], 
    ['Baseline I', '11 Hz'], 
    ['Baseline IIa', 'Baseline IIb'], 
    ['Baseline II', '30 Hz'],
    ['Baseline I', 'Baseline II']
]

## compute baseline values

In [18]:
def compute_baseline(column):
    total_itr = sum([results[cell_type]['action'][column].count() for cell_type in cell_types])
    pbar = tqdm(total=total_itr)

    for cell_type in cell_types:

        for idx, action_id in results[cell_type]['action'].loc[:,column].dropna().items():
            unit_name, channel_group = results[cell_type]['unit_name'].loc[idx, [column, 'channel_group']]
            rate_map_1, rate_map_2 = data_loader.rate_map_split(
                        action_id, channel_group, unit_name, smoothing_low)

            spikes_1, spikes_2, t_split = data_loader.spike_train_split(
                        action_id, channel_group, unit_name)

            t1, v1, t2, v2 = map(data_loader.tracking_split(action_id).get, ['t1', 'v1', 't2', 'v2'])

            pbar.update()
            for spikes, rate_map, speed, times, label in zip([spikes_1, spikes_2], [rate_map_1, rate_map_2], [v1, v2], [t1, t2], [column + 'a', column + 'b']):
                results[cell_type]['gridness'].loc[idx, label] = sp.gridness(rate_map)

                results[cell_type]['max_rate'].loc[idx, label] = rate_map.max()

                results[cell_type]['average_rate'].loc[idx, label] = len(spikes) / t_split

                results[cell_type]['speed_score'].loc[idx, label] = spd.speed_correlation(
                    speed, times, spikes, return_data=False)
    pbar.close()

In [19]:
compute_baseline('Baseline I')

In [20]:
compute_baseline('Baseline II')

In [67]:
def compute_changing_statistics(results, labels, key):
    df = results[key]
    if key == 'gridness':
        values = df[labels[0]] - df[labels[1]]
    elif key == 'speed_score':
        values = df[labels[0]] - df[labels[1]]
    elif key == 'max_rate':
        values = (df[labels[0]] - df[labels[1]]) / df[labels[0]]
    elif key == 'average_rate':
        values = (df[labels[0]] - df[labels[1]]) / df[labels[0]]
    elif key == 'xcorr_cntr_mass':
        values = pd.Series(index=df.index)
        for idx, row in results['action'].loc[:, labels].dropna().iterrows():
            action_id_1, action_id_2 = row.loc[labels]
            unit_name_1, unit_name_2, channel_group = results['unit_name'].loc[idx, labels + ['channel_group']]

            rate_map_1 = data_loader.rate_map(
                action_id_1, channel_group, int(unit_name_1), smoothing_low)
            
            rate_map_2 = data_loader.rate_map(
                action_id_2, channel_group, int(unit_name_2), smoothing_low)
            distance, angle = cross_correlation_centre_of_mass(rate_map_1, rate_map_2)
            values[idx] = distance * bin_size
    else:
        raise ValueError(key)
    
    return values

In [72]:
results_change = {}
for cell_type in tqdm(cell_types):
    results_change[cell_type] = {}
    for key in tqdm(results[cell_type].keys(), desc=cell_type):
        if key in ['action', 'unit_name']:
            continue
        results_change[cell_type][key] = pd.DataFrame()
        for diff_label in diff_labels:
            results_change[cell_type][key][' - '.join(diff_label)] = compute_changing_statistics(results[cell_type], diff_label, key)
            results_change[cell_type][key].loc[:, 'entity'] = results[cell_type][key].loc[:, 'entity']
            results_change[cell_type][key].loc[:, 'date'] = results[cell_type][key].loc[:, 'date']
            results_change[cell_type][key].loc[:, 'unit_idnum'] = results[cell_type][key].loc[:, 'unit_idnum']

# save to output folder

In [73]:
for cell_type, value_holder in results_change.items():
    for key, df in value_holder.items():
        path = output_path / "data" / "results_change" / cell_type
        path.mkdir(exist_ok=True, parents=True)
        df.to_csv(path / (key + '.csv'))

        
for cell_type, value_holder in results.items():
    for key, df in value_holder.items():
        path = output_path / "data" / "results_unique_cells" / cell_type
        path.mkdir(exist_ok=True, parents=True)
        df.to_csv(path / (key + '.csv'))

# Plotting raw data

In [28]:
plt.rc('axes', titlesize=12)
plt.rcParams.update({
    'font.size': 12, 
    'figure.figsize': (6, 4), 
    'figure.dpi': 150
})

In [29]:
def make_plot(what, axs, action_ids, unit_names, channel_group):
    if what == 'rate_map':
        sns.despine(left=True, bottom=True)
        for ax, action_id, unit_name in zip(axs, action_ids, unit_names):
            if np.isnan(unit_name):
                continue
            vmax = None
            rate_map = data_loader.rate_map(action_id, channel_group, int(unit_name), smoothing_low)
            if vmax is None:
                vmax = rate_map.max()
            ax.imshow(rate_map, origin='lower', vmax=vmax)
    #             ax.set_title(f'{row.gridness:.2f} {row.max_rate:.2f} {row.average_rate:.2f}')
            ax.set_yticklabels([])
            ax.set_xticklabels([])
    if what == 'spike_map':
        from scipy.interpolate import interp1d
        for ax, action_id, unit_name in zip(axs, action_ids, unit_names):
            if np.isnan(unit_name):
                continue
            x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])
            ax.plot(x, y, 'k', alpha=0.3)
            spike_times = data_loader.spike_train(action_id, channel_group, unit_name)
            spike_times = spike_times[(spike_times > min(t)) & (spike_times < max(t))]
            x_spike = interp1d(t, x)(spike_times)
            y_spike = interp1d(t, y)(spike_times)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.scatter(x_spike, y_spike, marker='.', color=(0.7, 0.2, 0.2), s=1.5)
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            ax.set_aspect(1)
    if what == 'speed':
        colors = ['#1b9e77','#d95f02','#7570b3', '#e7298a']
        for ax, action_id, unit_name, color in zip(axs, action_ids, unit_names, colors):
            if np.isnan(unit_name):
                continue
            x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])

            spike_times = data_loader.spike_train(action_id, channel_group, unit_name)
            spike_times = spike_times[(spike_times > min(t)) & (spike_times < max(t))]
        
            speed_score, inst_speed, rate, times = spd.speed_correlation(
                speed, t, spike_times, return_data=True)

            speed_bins = np.arange(min_speed, max_speed + speed_binsize, speed_binsize)
            ia = np.digitize(inst_speed, bins=speed_bins, right=True)
            rates = []

            for i in range(len(speed_bins)):
                rates.append(rate[ia==i])

            ax.set_title(f'{speed_score:.3f}')
            plot_bootstrap_timeseries(speed_bins, rates, ax=ax, normalize_values=True, color=color)
#             rr = [rr for r in rates for rr in r]
#             aspect = (max_speed - min_speed) / (np.nanmax(rr) - np.nanmin(rr))
            ax.set_aspect('auto')

In [30]:
pure_labels = [
    'Baseline I', 
    '11 Hz', 
    'Baseline II', 
    '30 Hz'
]

def plot(what, cell_type):

    n_neurons = len(results[cell_type]['action'])
    fig, axss = plt.subplots(n_neurons, 4, sharey=True, sharex=True, figsize=(8, 2*n_neurons))

    for (idx, row), axs in zip(results[cell_type]['action'].iterrows(), axss):
        action_ids = row.loc[pure_labels].values
        unit_names = results[cell_type]['unit_name'].loc[idx, pure_labels].values
        channel_group, unit_idnum, entity, date = row.loc[['channel_group', 'unit_idnum', 'entity', 'date']]
        axs[0].set_ylabel(f'{unit_idnum} {entity}-{date}')
        make_plot(what, axs, action_ids, unit_names, channel_group)
    plt.tight_layout()

    # fig.savefig(output_path / 'figures' / f'neuron_{id_num}_rate_map.png', bbox_inches='tight')
    # savefig(output_path / 'figures' / f'{what}-{cell_type}')
    fig.savefig(output_path / 'figures' / f'{what}-{cell_type}.svg', bbox_inches='tight')

In [36]:
%%capture
plot(cell_type='gridcell', what='rate_map')
plot(cell_type='gridcell', what='speed')
plot(cell_type='ns_inhibited', what='speed')

# Save to expipe

In [31]:
action = project.require_action("longitudinal-comparisons")

In [32]:
copy_tree(output_path, str(action.data_path()))

['/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/paired-statistics_ns_not_inhibited.tex',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/paired-statistics_gridcell.tex',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/statistics_gridcell.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/paired-statistics_ns_inhibited.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/statistics_ns_inhibited.tex',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/statistics_bs.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/paired-statistics_bs_not_gridcell.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/statistics/statistics_gridcell.tex',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/st

In [33]:
septum_mec.analysis.registration.store_notebook(action, "10_longitudinal_comparisons.ipynb")